In [11]:
import numpy as np
import os

from pathlib import Path

# change working directory to make src visible
os.chdir(Path.cwd().parent)

from src.data_generation import SPDEventGenerator
from src.visualization import draw_event

In [12]:
event_gen = SPDEventGenerator(detector_eff=0.98)

for _ in range(2):
    event = event_gen.generate_spd_event()
    print(event)

Event:
Shape of real hits: (345, 3)
Shape of momentums: (345, 3)
Shape of fake hits: (2486, 3)
Fraction of fakes: 0.88
Fraction of missing hits: 0.01
Number of unique tracks: 10
Vertex: Vertex(x=-5.85, y=-20.63, z=213.35)
Track parameters:
	Track ID: 0, TrackParams(pt=397.89, phi=1.07, theta=0.90, charge=-1)
	Track ID: 1, TrackParams(pt=746.35, phi=2.47, theta=1.61, charge=1)
	Track ID: 2, TrackParams(pt=148.99, phi=5.69, theta=1.35, charge=1)
	Track ID: 3, TrackParams(pt=399.82, phi=2.74, theta=2.35, charge=1)
	Track ID: 4, TrackParams(pt=907.12, phi=1.85, theta=2.41, charge=1)
	Track ID: 5, TrackParams(pt=587.82, phi=1.23, theta=1.86, charge=-1)
	Track ID: 6, TrackParams(pt=605.35, phi=3.77, theta=1.70, charge=-1)
	Track ID: 7, TrackParams(pt=392.64, phi=5.30, theta=2.03, charge=-1)
	Track ID: 8, TrackParams(pt=696.15, phi=5.69, theta=2.66, charge=1)
	Track ID: 9, TrackParams(pt=651.80, phi=2.07, theta=0.70, charge=1)


Event:
Shape of real hits: (346, 3)
Shape of momentums: (346, 3)

In [13]:
draw_event(
    hits=event.hits,
    fakes=event.fakes,
    vertex=event.vertex.numpy,
    labels=event.track_ids,
)

In [14]:
event.track_params

{0: TrackParams(phi=4.5371810014531215, theta=1.9960917473054214, pt=145.2302989782409, charge=-1),
 1: TrackParams(phi=3.774428451681594, theta=1.7645019685287977, pt=734.6552614916181, charge=-1),
 2: TrackParams(phi=0.9808151177817322, theta=0.37798978348329726, pt=828.4409027613177, charge=-1),
 3: TrackParams(phi=6.196660181136958, theta=2.7196557499212113, pt=984.45165559925, charge=-1),
 4: TrackParams(phi=5.32107872771094, theta=1.5253319181975955, pt=255.9737063744156, charge=1),
 5: TrackParams(phi=5.79424224353368, theta=2.3890580357418605, pt=556.3091244736676, charge=-1),
 6: TrackParams(phi=2.363169445477594, theta=1.1053766634786648, pt=386.30919821538515, charge=1),
 7: TrackParams(phi=0.08149950960544734, theta=2.6542814913577084, pt=728.0882299886016, charge=-1),
 8: TrackParams(phi=3.195930982460484, theta=2.6223859992757586, pt=684.0702160522378, charge=-1),
 9: TrackParams(phi=5.982906146525647, theta=1.3318070833001323, pt=352.5826412662945, charge=1)}

In [15]:
event.vertex

Vertex(x=6.891454918950769, y=-1.3271993317644173, z=-31.939593887449917)

## Example of reproducing tracks by their parameters

In [16]:
hits, labels = [], []
for track, track_params in event.track_params.items():
    track_hits = event_gen.reconstruct_track_hits_from_params(track_params, event.vertex)
    hits.append(track_hits)
    labels.extend([track]*len(track_hits))
    
hits = np.vstack(hits, dtype=np.float32)
labels = np.array(labels, dtype=np.int32)

In [17]:
draw_event(
    hits=hits,
    fakes=None,
    vertex=event.vertex.numpy,
    labels=labels,
)

## Visualize normalized hits

In [18]:
from src.normalization import ConstraintsNormalizer

normalizer = ConstraintsNormalizer()

draw_event(
    hits=normalizer(event.hits),
    fakes=normalizer(event.fakes),
    vertex=normalizer(np.expand_dims(event.vertex.numpy, 0)).squeeze(0),
    labels=event.track_ids,
    x_coord_range=[-1, 1],
    y_coord_range=[-1, 1],
    z_coord_range=[-1, 1],
)